In [1]:
#data scource: 
# read data 
import gzip
import json
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

In [ ]:
# import itertools
# def parse(path):
#     g = gzip.open(path,'r')
#     for l in g:
#         yield json.loads(l)
        
# # chunk a large iterator
# def chunks(iterable, size=1000):
#     iterator = iter(iterable)
#     for first in iterator:
#         yield itertools.chain([first], itertools.islice(iterator, size - 1))
        
# # iterate generator into csv file        
# def get_dataframe(path):
#     i = 0
#     df = {}
#     for d in parse(path):
#         df[i] = d
#         i += 1
#     return pd.DataFrame.from_dict(df, orient = 'index')

# def get_dataframe2(gen):
#     i = 0
#     df = {}
#     for d in gen:
#         df[i] = d
#         i += 1
#     return pd.DataFrame.from_dict(df, orient = 'index')

# # write generator into small chunks and into different df
# for i, single_gen in enumerate(chunks(parse(r'C:\Users\mengx\Desktop\Python projects\recommendation_system\Sports_and_Outdoors.json.gz'), size=5000000)):
#     df = get_dataframe2(single_gen)
#     df_name = 'magzine_df_{}.csv'.format(i)
#     df.to_csv(df_name, index=True)
    
# # write the gzip file into a dataframe keeping 'overall','reviewTime','asin','reviewerID'
# def get_dataframe3(path):
#     i = 0
#     df = {}
#     for d in parse(path):
#         df[i] = [d['overall'],d['reviewTime'],d['reviewerID'],d['asin']]
#         i += 1
#     return pd.DataFrame.from_dict(df, orient = 'index')


# 1. Collaborative filtering

In [353]:
# load data
df_scores = pd.read_csv(r'C:\Users\mengx\Desktop\Python projects\recommendation_system\Sports_4_col.csv', header=0, names = ['score','time', 'user','product'])
df_scores.drop_duplicates(subset=['user','product'], inplace=True)

In [354]:
def k_core(df_scores, k):
    
    temp_list1_product = df_scores['product'].value_counts()[df_scores['product'].value_counts()>=k].index.tolist()
    temp_list2_user = df_scores['user'].value_counts()[df_scores['user'].value_counts()>=k].index.tolist()
    df_scores_short = df_scores[(df_scores['product'].isin(temp_list1_product)) & (df_scores['user'].isin(temp_list2_user))]
    
    df_scores_short.head()
    df_scores_short.loc[:,'time'] = pd.to_datetime(df_scores_short['time'])
    df_scores_short.head()
    
    print('Oringal size of the df: ', df_scores.shape)
    print('# of users: ', df_scores['user'].nunique())
    print('# of products: ', df_scores['product'].nunique())
    print('Sparsity before: ', df_scores.size/(df_scores['user'].nunique()*df_scores['product'].nunique()))
    print('After keeping {} more reviews on products and users size of the df: {}'.format(k, df_scores_short.shape))
    print('# of users: ', df_scores_short['user'].nunique())
    print('# of products: ', df_scores_short['product'].nunique())
    print('Sparsity after:',df_scores_short.size/(df_scores_short['user'].nunique()*df_scores_short['product'].nunique()))
    
    return df_scores_short

In [107]:
#df_scores_short.sort_values(by='time', inplace=True)

C:\Softwares\Anaconda_install\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
# #train test split
# # use the first 80% data of each user in time stamp as the train, the rest as test
# df_scores_short_train = pd.DataFrame(columns=['score','time','user','product'])
# df_scores_short_test = pd.DataFrame(columns=['score','time','user','product'])
# i = 0
# for user in temp_list2:
#     temp_len = df_scores_short[df_scores_short['user']==user].shape[0]
#     temp_train = df_scores_short[df_scores_short['user']==user].iloc[0:int(temp_len*0.8)]
#     temp_test = df_scores_short[df_scores_short['user']==user].iloc[int(temp_len*0.8):]
#     df_scores_short_train = pd.concat([df_scores_short_train, temp_train])
#     df_scores_short_test = pd.concat([df_scores_short_test, temp_test])
#     i+=1
#     if i%500 ==1 :
#         print(i)
    

1
501
1001
1501
2001
2501
3001
3501
4001
4501
5001
5501
6001
6501
7001
7501
8001
8501
9001
9501
10001
10501
11001


In [2]:
# df_scores_short_train = pd.read_csv('df_scores_4_col_short_25core_train.csv')
# df_scores_short_test = pd.read_csv('df_scores_4_col_short_25core_test.csv')
# df_scores_short = pd.concat([df_scores_short_train,df_scores_short_test])

In [355]:
df_scores_short = k_core(df_scores, k=25)
#df_scores_short_pivot = df_scores_short.pivot(index='user',columns='product', values='score')

C:\Softwares\Anaconda_install\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Oringal size of the df:  (12601954, 4)
# of users:  6703391
# of products:  957764
Sparsity before:  7.851358723155962e-06
After keeping 25 more reviews on products and users size of the df: (313236, 4)
# of users:  11474
# of products:  56038
Sparsity after: 0.0019486515546613913


In [2]:
# load small dataset for test
df_scores_short_random_100K = pd.read_csv('df_scores_short_random_100K.csv')
#df_scores_short_random_10K = pd.read_csv('df_scores_short_random_10K.csv')

In [5]:
# fill pivot dataframes with mean
def pivot_fillmean(df):
    start_time = time.time()
    df_pivot = df.pivot(index='user',columns='product', values='score')
    df = df.T.fillna(df.mean(axis=1)).T      # not minus mean
    mean_df = df_pivot.mean(axis=1)
    #df_pivot = (df_pivot.T.fillna(df_pivot.mean(axis=1))-df_pivot.mean(axis=1)).T        #minus mean
    print('Time eclapse for pivot and fill number with mean:', time.time()-start_time)
    return df_pivot, mean_df
df_short_random_ready, mean_df = pivot_fillmean(df_scores_short_random_100K)

NameError: name 'df_scores_short_random_100K' is not defined

In [15]:
def sin_value_decomposition(df, k):
    from scipy.sparse.linalg import svds
    import time
    start_time = time.time()
    U, sigma, Vt = svds(df, k = k)
    print('running time for SVD: ', time.time()-start_time)
    sigma_matrix = np.diag(sigma)
    pred_mat = np.dot(np.dot(U,sigma_matrix),Vt)
    pred_df = pd.DataFrame(pred_mat, columns=df.columns, index=df.index)
    print('RMS error:', sqrt(mean_squared_error(pred_df, df)))
    return pred_df

In [16]:
pred_df = sin_value_decomposition(df_short_random_ready, 50)

running time for SVD:  151.23677110671997
RMS error: 0.01253096052585067


In [24]:
def add_mean_back(pred_df, mean_df):
    pred_df = (pred_df.T + mean_df).T
    return pred_df

pred_df = add_mean_back(pred_df, mean_df)

In [126]:
def get_reco(x, df, original_df, N):
    recomm_df_temp_1 = pd.DataFrame(pred_df.loc[x].sort_values(ascending=False).reset_index())
    recomm_df_temp_1.columns=['product', 'scores']

    recomm_df_temp_2 = recomm_df_temp_1.head(100)
    recomm_df_temp_3 = recomm_df_temp_1[~recomm_df_temp_1['product'].isin(original_df[original_df.user==x]['product'].values)].head(N)
    return recomm_df_temp_2, recomm_df_temp_3

reco_df_Col_based = get_reco('A2K9S5LU6EH87R', pred_df, df_scores_short_random_100K, 10)[0]

In [131]:
reco_df_Col_based.head(20)

,product,scores
0,B000IZGFCE,4.009939
1,B002AGI1QW,4.009699
2,B00A3W8EIU,4.006841
3,B002DL4KCS,4.006516
4,B001BSQCWU,4.006289
5,B000OZDIGY,4.005924
6,B00HNB7AUC,4.005860
7,B009IF2M3G,4.005615
8,B0002YTO7E,4.005264
9,B00KQSVRF8,4.005192


In [89]:
pred_df

product,0899333257,7245456275,7245456313,7500670133,B000002NUS,B000002NUT,B00000IT1G,B00002N6T4,B00002N9FI,B00003G4JR,...,B01HI69CB6,B01HIGRIEE,B01HIIQ050,B01HILPQLQ,B01HIN1KUU,B01HIUP6YY,B01HJ4DN08,B01HJA7KWE,B01HJDGJ1E,B01HJDHNX2
user,,,,,,,,,,,,,,,,,,,,,
A0029274J35Q1MYNKUWO,4.499997,4.500000,4.499946,4.500000,4.499904,4.499984,4.500000,4.499999,4.500000,4.500000,...,4.500000,4.500000,4.500000,4.499977,4.500029,4.500000,4.499998,4.500000,4.500018,4.499996
A0098869QS7I6WOAODWI,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
A0464351OZXPUPKGI6HO,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
A0784213CF0H1GDXG0G2,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
A07936821FOVJO6NP4Q8,4.600000,4.600000,4.600023,4.600000,4.600018,4.600007,4.600000,4.600001,4.600000,4.600000,...,4.600000,4.600000,4.600000,4.599996,4.600015,4.600000,4.600000,4.600000,4.600011,4.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZY9H78K5ZK52,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
AZYJE40XW6MFG,4.222221,4.222222,4.222139,4.222222,4.222193,4.222166,4.222222,4.222222,4.222222,4.222221,...,4.222222,4.222221,4.222222,4.222219,4.222207,4.222222,4.222222,4.222222,4.222208,4.222226
AZYJMUE8PNTQV,4.749988,4.750000,4.748521,4.750000,4.749698,4.749832,4.750000,4.749990,4.750002,4.750001,...,4.749982,4.750007,4.750001,4.749785,4.750079,4.750000,4.749993,4.750000,4.750017,4.749977


# 2. Content-based filtering

In [4]:
# get product meta data
import itertools
def parse(path):
    g = gzip.open(path,'r')
    for l in g:
        yield json.loads(l)
       
# iterate generator into csv file 
temp_list = df_scores_short_random_100K['product'].values
def get_product_information(path):
    i = 0
    df = {}
    for d in parse(path):
        if d['asin'] in temp_list:
            df[i] = d
            i += 1
            if i%1000 == 0:
                print(i)
        else:
            continue
    return pd.DataFrame.from_dict(df, orient = 'index')

product_meta = get_product_information('meta_Sports_and_Outdoors.json.gz')
product_meta = product_meta[['category','price','asin','title','description','feature']]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000


In [150]:
#aa = parse('meta_Sports_and_Outdoors.json.gz')
#next(aa)

In [31]:
product_meta = pd.read_pickle('product_meta_100K.pickle')
product_meta.head(5)

,category,price,asin,title,description,feature
0,"[Sports & Outdoors, Outdoor Recreation, Campin...",$21.96,0899333257,Garmin DeLorme Atlas & Gazetteer Paper Maps- A...,[Rely on delorme ATLAS & gazetteer paper maps ...,"[Amazingly detailed and beautifully crafted, l..."
1,"[Sports & Outdoors, Sports & Fitness, Exercise...",$17.32,7245456313,Black Mountain Products Resistance Band Set wi...,[Black Mountain Products (B.M.P.) resistance b...,"[Bands included: Yellow (2-4 lbs.), blue (4-6 ..."
2,"[Sports & Outdoors, Sports & Fitness, Exercise...",$14.77,7245456275,Black Mountain Products Single Resistance Band...,[Black Mountain Products single resistance ban...,"[1 Blue band (4-6Lbs.), Attached foam handles...."
3,"[Sports & Outdoors, Sports & Fitness, Team Spo...",$4.99,7500670133,Titanium Tornado Baseball Necklace Orange Blac...,[Materials emit energy that is effective in co...,NaN
4,"[Sports & Outdoors, Sports & Fitness, Hunting ...",NaN,B000002NUT,Outers 70074 Universal 62-Piece Blow Molded Gu...,[Outers 62 piece blow molded 70074 gun cleanin...,"[Made of the highest quality materials, Huntin..."


In [32]:
def category_process(x):
    if isinstance(x, list):
        temp = ' '.join([i.lower().replace('&',' ').replace('   ',' ').replace('  ',' ') for i in x])
        return ' '.join(set(temp.split()))
    elif isinstance(x, str):
        return x.lower()
    else:
        return ''

import string

def description_process(x):
    if type(x) == float:
        temp = ''
    else:
        x = list(x)
        temp = ' '.join([i.lower() for i in x])
        temp = ''.join([i for i in temp if i not in frozenset(string.punctuation)])
    return temp

In [33]:
product_meta['category_strip'] = product_meta['category'].apply(category_process)
product_meta['description_strip'] = product_meta['description'].apply(description_process)
product_meta['feature_strip'] = product_meta['feature'].apply(description_process)
product_meta['cate_desc_fea_strip'] = product_meta['category_strip'] + product_meta['description_strip'] + product_meta['feature_strip']

In [214]:
#product_meta.reset_index(inplace=True)

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(product_meta['cate_desc_fea_strip'])

In [118]:
# get simulatrity matrix

from sklearn.metrics.pairwise import linear_kernel

def get_sim(x, df, N):   # x is user name  df is dataframe N is the top N recommendations
    product_list = df[df['user'] == x]['product'].tolist() # product list of a certain user, index
    product_list_total = product_meta['asin'].tolist()   #columns
    product_list_idx = list(int(product_meta[product_meta['asin'] == i].index.values) for i in product_list)
    
    correlation = linear_kernel(tfidf_matrix[product_list_idx,:], tfidf_matrix)
    # set self correlation to zero
    for a in range(len(product_list_idx)):
        correlation[a, product_list_idx[a]]=0
    
    # top N product for a certain user
    idx = np.argsort(correlation.ravel())[-100:][::-1]
    topN_val = correlation.ravel()[idx]
    row_col = np.c_[np.unravel_index(idx, correlation.shape)]
    
    CB_product_df = pd.DataFrame(topN_val, index = [product_list_total[row_col[i,1]] for i in range(len(topN_val))], columns=['Content_Based'])
    CB_product_df.reset_index(inplace=True)
    CB_product_df.columns=['asin','Content_Based_score']
    CB_product_df = pd.merge(CB_product_df, product_meta[product_meta['asin'].isin(CB_product_df['asin'].values)], on='asin', how='left')
#     print('UserID:', x)
#     print('************')
    bought_product = product_meta[product_meta['asin'].isin(df[df['user'] == x]['product'].values)]
#     print('User bought:', bought_product['title'].values)
#     print('************')
#     print('suggested:' )
#     print(CB_product_df[['Content_Based_score','title','price']])

    return CB_product_df[['asin','Content_Based_score','title','price','category']], CB_product_df[['asin','Content_Based_score','title','price','category']].head(N)
    

In [128]:
reco_df_Content_based = get_sim('A2K9S5LU6EH87R', df_scores_short_random_100K, 10)[0]

In [130]:
reco_df_Content_based.head(20)

,asin,Content_Based_score,title,price,category
0,B0055C2NEE,0.860635,Rico Industries NFL LED Power Decal,$34.99,"[Sports & Outdoors, Fan Shop, Auto Accessories..."
1,B0013AW8D8,0.735990,Ontario 8604 RAT-7 Knife (Tan),NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ..."
2,B001EDOCDI,0.733726,Ontario 8632 RAT-3 Knife (Green),NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ..."
3,B001DZV3C0,0.725773,Ontario 8630 RAT-3 Knife,NaN,"[Sports & Outdoors, Outdoor Recreation, Campin..."
4,B00LMS11CA,0.722669,TALON Grips for Glock 42,$17.99,"[Sports & Outdoors, Sports & Fitness, Hunting ..."
5,B000R30XAC,0.722249,Ontario Knife 1086277 Co RAT-5 1095 Knife,NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ..."
6,B000RUZ9MM,0.693131,Rico NCAA Small Static Decal,NaN,"[Sports & Outdoors, Fan Shop, Home & Kitchen, ..."
7,B0018PG10E,0.674414,Ontario Spec Plus Gen II SP43 Knife (Black),NaN,"[Sports & Outdoors, Outdoor Recreation, Campin..."
8,B00ZHJ8DYE,0.669245,"BFR BANDS Occlusion Training Bands by, Elite M...",NaN,"[Sports & Outdoors, Sports & Fitness, Exercise..."
9,B003C1HEBC,0.649971,Ontario Spec Plus Gen II SP50 Knife (Black),NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ..."


In [155]:
def combine_Col_Content(x, SVD_df, original_df, N):
    Col_based = get_reco(x, SVD_df, original_df, N)[0]
    Col_based['logit_col_scores'] = Col_based['scores'].apply(lambda x: 1/(1+2.718**-x))
    Content_based = get_sim(x, original_df, N)[0]
    Content_based['logit_col_scores'] = Content_based['Content_Based_score'].apply(lambda x: 1/(1+2.718**-x))
    
    return Col_based, Content_based
    

In [156]:
combine_Col_Content('A2K9S5LU6EH87R', pred_df, df_scores_short_random_100K, 10)[1]

,asin,Content_Based_score,title,price,category,logit_col_scores
0,B0055C2NEE,0.860635,Rico Industries NFL LED Power Decal,$34.99,"[Sports & Outdoors, Fan Shop, Auto Accessories...",0.702775
1,B0013AW8D8,0.735990,Ontario 8604 RAT-7 Knife (Tan),NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ...",0.676102
2,B001EDOCDI,0.733726,Ontario 8632 RAT-3 Knife (Green),NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ...",0.675606
3,B001DZV3C0,0.725773,Ontario 8630 RAT-3 Knife,NaN,"[Sports & Outdoors, Outdoor Recreation, Campin...",0.673861
4,B00LMS11CA,0.722669,TALON Grips for Glock 42,$17.99,"[Sports & Outdoors, Sports & Fitness, Hunting ...",0.673178
...,...,...,...,...,...,...
95,B013TGT2ZU,0.308586,Tackedoo PRO Scooter Fully Integrated - HIC Co...,NaN,"[Sports & Outdoors, Outdoor Recreation, Skates...",0.576532
96,B0015NDA1C,0.307035,var aPageStart = (new Date()).getTime();\nvar ...,$13.22,"[Sports & Outdoors, Sports & Fitness, Hunting ...",0.576154
97,B014Z11GEI,0.304753,"GW Geek Waterproof 6.5"" 8"" 10"" Smart Two Wheel...",NaN,"[Sports & Outdoors, Outdoor Recreation, Skates...",0.575596
98,B000AU1RTS,0.304744,"Hogue 45009 Colt & 1911 Officer's Grip Screws,...",NaN,"[Sports & Outdoors, Sports & Fitness, Hunting ...",0.575594


In [157]:
combine_Col_Content('A2K9S5LU6EH87R', pred_df, df_scores_short_random_100K, 10)[0]

,product,scores,logit_col_scores
0,B000IZGFCE,4.009939,0.982181
1,B002AGI1QW,4.009699,0.982177
2,B00A3W8EIU,4.006841,0.982127
3,B002DL4KCS,4.006516,0.982121
4,B001BSQCWU,4.006289,0.982117
...,...,...,...
95,B002SRDG72,4.001640,0.982035
96,B00WLFT6HK,4.001635,0.982035
97,B000VZ1IJG,4.001633,0.982035
98,B016EE9P1K,4.001632,0.982035
